In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('C:/Users/USER/Downloads/kobe.csv')
shot,time,date,date1=[],[],[],[]
for i in range(len(df)):
    date+=[str(df['game_date'][i])]
for i in range(len(df)):
    l,r=date[i].find('/'),date[i].rfind('/')
    if (r-l==2) and (len(date[i][r+1:])==1):
        date1+=[int(str(date[i][:l])+'0'+str(date[i][l+1:r])+'0'+str(date[i][r+1:]))]
    elif (r-l==2) and (len(date[i][r+1:])!=1):
        date1+=[int(str(date[i][:l])+'0'+str(date[i][l+1:r])+str(date[i][r+1:]))]
    elif (r-l==3) and (len(date[i][r+1:])==1):
        date1+=[int(str(date[i][:l])+str(date[i][l+1:r])+'0'+str(date[i][r+1:]))]
    elif (r-l==3) and (len(date[i][r+1:])!=1):
        date1+=[int(str(date[i][:l])+str(date[i][l+1:r])+str(date[i][r+1:]))]
    time+=[df['minutes_remaining'][i]+(df['seconds_remaining'][i]/100)]
    if df['shot_type'][i]=='2PT Field Goal':
        shot+=[2]
    elif df['shot_type'][i]=='3PT Field Goal':
        shot+=[3]
df['shot_type']=pd.Series(shot)
df['time_leave']=pd.Series(time)
df['date1']=pd.Series(date1)

# 1

In [2]:
opp=df['opponent']=='HOU'
def in_(a):
    mask=df[opp]['shot_type']==a
    print("two: %.2f"%(df[opp][mask]['shot_made_flag'].sum()/len(df[opp][mask])))
in_(2)
in_(3)

two: 0.39
two: 0.27


# 2

In [3]:
name=df['opponent'].unique()
times,score=[],[]
for i in name:
    mas=df['opponent']==i
    game_id=df[mas]['game_id'].unique()
    times+=[len(game_id)]
    scores=0
    for ii in df.index[df['opponent']==i]:
        scores+=(df['shot_made_flag'][ii]*df['shot_type'][ii])
    score+=[scores]
data=pd.DataFrame({'opponent':name,'score':score,'times':times})
data['平均得分']=round(data['score']/data['times'],3)
data=data.sort_values(by='平均得分')
data[['opponent','平均得分']].head()

,opponent,平均得分
2,VAN,11.167
32,BKN,12.667
11,IND,14.026
1,UTA,14.143
21,NJN,14.179


# 3

In [4]:
opp=df['opponent'].unique()
mask=df['time_leave']<3
mask1=df[mask]['period']==4
mask2=df[mask][mask1]['playoffs']==1
result=pd.DataFrame(columns=['opponent','game_id','得分'])
for k in opp:
    result1=pd.DataFrame(columns=['opponent','game_id','得分'])
    get=[]
    oppo=df[mask][mask1][mask2]['opponent']==k
    game_iid=df[mask][mask1][mask2][oppo]['game_id'].unique()
    for kk in game_iid:
        get_=0
        for kkk in df[mask][mask1][mask2].index[df[mask][mask1][mask2]['game_id']==kk]:
            get_+=(df[mask][mask1][mask2]['shot_made_flag'][kkk]*df[mask][mask1][mask2]['shot_type'][kkk])
        get+=[get_]
    for i in range(len(game_iid)):
        result1.loc[i]={'opponent':k,'game_id':game_iid[i],'得分':get[i]}
    result=pd.concat([result,result1],axis=0,ignore_index=True)
result=result.sort_values(by='得分',ascending=False)
result.head()

,opponent,game_id,得分
8,POR,49900074,9
42,SAS,40100234,8
82,SAC,49900024,7
131,BOS,40700402,7
5,POR,49600024,6


# 4

In [5]:
seasondata=pd.DataFrame(columns=['season','all','in'])
mac=df['action_type']=='Jump Shot'
mac1=df[mac]['period']==4
mac2=df[mac][mac1]['playoffs']==1
mac3=df[mac][mac1][mac2]['time_leave']<=1
season=df['season'][mac][mac1][mac2][mac3].unique()
for s in season:
    seasondata1=pd.DataFrame(columns=['season','all','in'])
    all_,in_=0,0
    sea=df[mac][mac1][mac2][mac3]['season']==s
    seasondata1.loc[0]={'season':s,'all':len(df[mac][mac1][mac2][mac3][sea]),'in':df[mac][mac1][mac2][mac3][sea]['shot_made_flag'].sum()}
    seasondata=pd.concat([seasondata,seasondata1],axis=0,ignore_index=True)
seasondata['命中率']=seasondata['in']/seasondata['all']
seasondata[['season','命中率']]

,season,命中率
0,Jan-00,0
1,1-Feb,0.5
2,2-Mar,0.333333
3,3-Apr,0.166667
4,5-Jun,0
5,6-Jul,0.5
6,7-Aug,0
7,8-Sep,0.142857
8,9-Oct,0.333333
9,10-Nov,0


# 5

In [6]:
data33=pd.DataFrame(columns=['opponent','game_id','date','date1','all','in'])
oppo=df['opponent'].unique()
for h in oppo:
    _all,_in=[],[]
    data_33=pd.DataFrame(columns=['opponent','game_id','date','date1','all','in'])
    oppo1=df['opponent']==h
    game_day=(df[oppo1]['game_id'].unique(),df[oppo1]['date1'].unique(),df[oppo1]['game_date'].unique())
    for hh in game_day[0]:
        game=df[oppo1]['game_id']==hh
        _all_,_in_=len(df[oppo1][game]),df[oppo1][game]['shot_made_flag'].sum()
        _all+=[_all_]
        _in+=[_in_]
    for hhh in range(len(game_day[0])):
        data_33.loc[hhh]={'opponent':h,'game_id':game_day[0][hhh],'date':game_day[2][hhh],'date1':game_day[1][hhh],'all':_all[hhh],'in':_in[hhh]}
    data33=pd.concat([data33,data_33],axis=0,ignore_index=True)
data33['命中率']=data33['in']/data33['all']
data33=data33.sort_values(by='date1').reset_index()
data33.head(280)
ans=data33.index[data33['命中率']>=0.33]
ans1,ans_in,length=[],[],[]
for h_ in range(len(ans)-1):
    if ans[h_+1]==ans[h_]+1:
        ans_in+=[ans[h_]]
    else:
        ans_in+=[ans[h_]]
        ans1+=[ans_in]
        ans_in=[]
for _h in range(len(ans1)):
    length+=[(len(ans1[_h]),ans1[_h])]
length.sort(reverse=True)
for i in range(3):
    print('From',data33['date'][length[i][1][0]],'to',data33['date'][length[i][1][-1]],'場數',length[i][0])


From 2001/6/15 to 2001/12/7 場數 19
From 2000/2/1 to 2000/3/12 場數 19
From 2002/1/14 to 2002/2/17 場數 16


# 6

In [8]:
kdata=pd.DataFrame(columns=['opponent','game_id','date','first','second','all','in'])
oppo=df['opponent'].unique()
for r in oppo:
    kdata1=pd.DataFrame(columns=['opponent','game_id','date','first','second','all','in'])
    first,second,_all,_in=[],[],[],[]
    name=df['opponent']==r
    game_idd=(df[name]['game_id'].unique(),df[name]['game_date'].unique())
    for rr in game_idd[0]:
        first_,second_,all_,in_=0,0,0,0
        game_i=df[name]['game_id']==rr
        all_+=len(df[name][game_i])
        in_+=df[name][game_i]['shot_made_flag'].sum()
        for rrr in df[name][game_i].index[df[name][game_i]['period']<3]:
            first_+=(df[name][game_i]['shot_made_flag'][rrr]*df[name][game_i]['shot_type'][rrr])
        down=df[name][game_i]['period']>=3
        for rrr in df[name][game_i][down].index[df[name][game_i][down]['period']<=4]:
            second_+=(df[name][game_i][down]['shot_made_flag'][rrr]*df[name][game_i]['shot_type'][down][rrr])
        first+=[first_]
        second+=[second_]
        _all+=[all_]
        _in+=[in_]
    for i in range(len(game_idd[0])):
        kdata1.loc[i]={'opponent':r,'game_id':game_idd[0][i],'date':game_idd[1][i],'first':first[i],'second':second[i],'all':_all[i],'in':_in[i]}
    kdata=pd.concat([kdata,kdata1],axis=0,ignore_index=True)


In [9]:
kdata['命中率']=kdata['in']/kdata['all']
kdata['上下半場得分差']=kdata['first']-kdata['second']
kdata['該場得分']=kdata['first']+kdata['second']
res=kdata['first']>kdata['second']
ans=kdata[res].sort_values(by='命中率')
ans[['opponent','game_id','date','上下半場得分差','該場得分','命中率']].head(3).sort_values(by='命中率',ascending=False)

,opponent,game_id,date,上下半場得分差,該場得分,命中率
234,LAC,21501164,2016/4/5,2,2,0.0833333
1086,PHX,20300385,2003/12/21,2,2,0.0833333
662,GSW,21500214,2015/11/24,3,3,0.0714286


# 7

In [12]:
df['score']=df['shot_made_flag']*df['shot_type']
game=df['game_id'].unique()
oppo,date,score=[],[],[]
all_game=[]
for d in game:
    one_game,ddd=[],0
    dd=df['game_id']==d
    oppo+=[df[dd]['opponent'].unique()[0]]
    date+=[df[dd]['game_date'].unique()[0]]
    score+=[df[dd]['score'].sum()]
    index=list(df.index[df['game_id']==d])
    while ddd <len(index):
        i=index[ddd]
        if df[dd]['shot_made_flag'][i]==0:
            ddd+=1
            continue
        else:
            one_game+=[ddd]
            del index[:ddd+1]
            ddd=0
    m=max(one_game) if one_game else None
    if m!=None:
        all_game+=[m]
    else:
        all_game+=[0]
ans7=pd.DataFrame({'game_id':game,'連續失手球數':all_game,'日期':date,'對手':oppo,'得分':score})
ans7=ans7.sort_values(by='連續失手球數',ascending=False)
ans7.head(3)

,game_id,連續失手球數,日期,對手,得分
751,20901131,16,2010/4/2,UTA,8
887,21100769,15,2012/3/31,NOH,7
174,20200367,14,2002/12/19,NJN,10
